In [1]:
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import jieba
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from kafka import KafkaConsumer
from kafka import TopicPartition
import json
import time
import csv
import sched

In [2]:
def kafka_consumer(topic,offset):    
    news=[]
    consumer = KafkaConsumer(bootstrap_servers='192.168.99.100:9092')
    parti = TopicPartition(topic, 0)
    consumer.assign([parti])
    consumer.seek(parti,offset)
    for msg in consumer:
        print(msg.offset)
        print(msg.value.decode('utf-8'))
        if msg.value.decode('utf-8') == 'done':
            offset=msg.offset+1
            break
        news.append(msg.value.decode('utf-8'))
    return offset,news

In [3]:
def similar_analysis(f):
    corpus = []
    for line in f:
        corpus.append(" ".join(jieba.cut(line.split(',')[0], cut_all=False)))

    cv = CountVectorizer()
    term_doc = cv.fit_transform(corpus)

    vectorizer = TfidfVectorizer()
    tfidf = vectorizer.fit_transform(corpus)
    words = vectorizer.get_feature_names()

    transformer = TfidfTransformer()
    tfidf = transformer.fit_transform(term_doc)
    fv = tfidf.toarray()

    #for i in range(len(fv)):
        #print('----Document %d----' % (i))
        #print(fv[i])

    sort_list = []
    for i in range(len(fv)):
        for j in range(i,len(fv)):
            if i != j:
                c_s = cosine_similarity([fv[i]], [fv[j]])
                #if c_s[0] != 0:
                #if i==0 or j ==0:
                #print('Doc ',i,' and ',j,' cosine similarity : ',c_s[0])
                sort_list.append([i,j,c_s[0]])
                sort_list.append([j,i,c_s[0]])
    sort_list = sorted(sort_list, reverse=False,key = lambda x : x[0])
    count=-1
    total=[]
    for i in range(len(sort_list)):
        if sort_list[i][0] != count:
            count=count+1
            total.append([sort_list[i]])
        else:
            total[count].append(sort_list[i])
    result=[]
    for i in range(len(fv)):
        total[i] = sorted(total[i], reverse=True,key = lambda x : x[2])
        temp=[str(i)]
        for j in range(5):
            #print(total[i][j])
            temp.append((str(total[i][j][1])+'('+str(total[i][j][2][0])+')'))
        result.append(temp)
    """for i in range(len(sort_list)):
        print(sort_list[i])
        firsts = []
        seconds = []
        for j in range(len(words)):
            if tfidf[sort_list[i][0],j] > 1e-5:
                firsts.append(words[j])
            if tfidf[sort_list[i][1],j] > 1e-5:
                seconds.append(words[j])
        print(sort_list[i][0],','.join(firsts))
        print(sort_list[i][1],','.join(seconds))"""
    #print(result)
    return result

In [4]:
def save_json(temp,filename):
    news=[]
    for i in range(len(temp)):
        news.append({
            'title': temp[i],
            'id': i
        }) 
    with open(filename, 'w', encoding='utf-8') as f:
            json.dump(news, f, indent=2, sort_keys=True, ensure_ascii=False)

In [5]:
def save_csv(result,filename):
    with open(filename, 'w', newline='') as csvfile:
        for i in range(len(result)):
            testwriter = csv.writer(csvfile)
            #print(result[i])
            testwriter.writerow(result[i])

In [6]:
def main(offset1,offset2,count):
    topic1='poli'
    topic2='ente'
    schedule = sched.scheduler ( time.time, time.sleep )
    
    offset1,news1=kafka_consumer(topic1,offset1)
    time.sleep(1)
    offset2,news2=kafka_consumer(topic2,offset2)
    time.sleep(1)
    save_json(news1,'poli_news'+str(count)+'.json')
    save_json(news2,'ente_news'+str(count)+'.json')
    result1=similar_analysis(news1)
    result2=similar_analysis(news2)
    save_csv(result1,'poli_result'+str(count)+'.csv')
    save_csv(result2,'ente_result'+str(count)+'.csv')
    
    count=count+1
    schedule.enter(10800,0,main,(offset1,offset2,count))  
    print('wait 3 hours...')
    schedule.run()

In [ ]:
offset1=0
offset2=0
count=1
main(offset1,offset2,count)

0
蔡總統親自主持記者會 介紹新任總統府秘書長陳菊
1
沙烏地暗示希望油價達每桶80美元
2
白宮：川普有權開除穆勒
3
歐元區經濟今年開局疲軟 歐洲央行：無須擔心
4
墨西哥總統大選官方打壓 獨立候選人敗部復活
5
國泰金終止馬國併購 顧立雄仍看好金融新南向
6
苗栗縣府：公告免指定建築線地區，建築開發有效率
7
蔡英文明在黨中央約見蘇貞昌 勸進參選新北市長
8
思念母親！見慶生合照 蔡總統眼眶泛紅
9
吳釗燮：友邦、美日聲援我以觀察員參與WHA
10
羅室長戰風向
11
【Yahoo論壇／翁履中】「玉山計畫」撒錢徵才？改變制度和心態才是關鍵
12
北韓人民會議 核武和對外政策引關注
13
徐國勇：葉俊榮任教時赴陸 非當官時
14
【Yahoo論壇／王皓平】賴清德院長夜宿一晚的遷都奇想
15
【Yahoo論壇／紀炫宇】走下正義與邪惡的擂台 才能找到台灣價值
16
【Live】陳菊接任總統府秘書長 蔡總統正式宣布
17
國內金融：9:35止，拆款市場隔夜拆款成交利率為0.183%，上漲0.004個百分點
18
台大校長案 潘文忠：管中閔未說明兼職疑義
19
陸首個經濟特區奠基人 改革先驅吳南生病逝
20
清涼直播維持人氣 台灣司法人權進步協會：建議直播內容以健康正面為導向
21
蔡英文：要求執政團隊傾巢而出 說明政績
22
華商爆賄賂 藉中國學生會「洗白」 過程宛如犯罪電影
23
避邊緣化 日外相罕見訪南韓提議題清單
24
俄提調查敘化武決議案 安理會表決未過
25
國內匯市：新台幣匯率今晨以29.155元開盤，升值3.5分
26
美提議調查敘化武 俄否決安理會決議案
27
國內匯市：台灣銀行早盤人民幣牌價，買入4.545，賣出4.707
28
國內財經：金融八法修法遭疑致金融犯罪所得難沒收，金管會澄清沒有沒收說法
29
國內財經：金管會放寬境外基金管理規範，最快7月初上路
30
國內金融：週三(11日)央行發行四種天期可轉讓定存單，回收資金
31
蔡英文的說法與事實不同　孫大千：可能得了「政治認知失調症」！
32
安理會闖關失利 川普化武案面臨抉擇
33
新聞內幕》布局2020憂無領頭羊 小英提前拱蘇戰新北內幕
34
201移民宣誓入籍 華女來美15年 終成新公民
35
安理會化武案立場分歧 美俄草案皆未過關
36
風評：管中閔「違法」兼職，葉俊榮「與有榮焉」
3

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LeoWang\AppData\Local\Temp\jieba.cache
Loading model cost 0.720 seconds.
Prefix dict has been built succesfully.


wait 3 hours...
